# Identify sales

In [7]:
import pandas as pd
from pathlib import Path
import csd as c

In [8]:
loc = Path.cwd().parent.parent / 'auxiliary' / 'data' / 'invoice_data_2_in-scope.csv'
inv_df = pd.read_csv(loc)

## All invoice lines of sales

## <span style="color:red"> Change vendor number(s)

In [9]:
sales_df = inv_df[ (inv_df['vendno'] == 18821) | (inv_df['vendno'] == 86125) ]

In [10]:
sales_df.to_clipboard()

In [11]:
sales_df.head()

,whse,orderno,ordersuf,lineno,invoicedt,custno,custname,item,itemdesc,unit,...,pdrecno,vendno,arpvendno,xcost_adj,GP$,Margin,month,Exclusion,whse_name,division
98,101,911007174,0,1,2022-03-04,100046,AIR PRO HEATING & COOLING,PVC500,"3""X10' PVC SCH40 PIPE",EA,...,67052,18821.0,0.0,207.15430,299.94570,0.591492,2022-03,NaN,Clifton HVAC,HVAC
118,101,911007696,0,6,2022-03-04,100046,AIR PRO HEATING & COOLING,PVC50A,"2""X10' PVC SCH40 PIPE",EA,...,9373,18821.0,0.0,21.32881,4.15119,0.162920,2022-03,NaN,Clifton HVAC,HVAC
546,101,911010559,0,1,2021-12-01,101252,TOTAL COMFORT HVAC LLC,PVC50A,"2""X10' PVC SCH40 PIPE",EA,...,9373,18821.0,0.0,107.01576,57.02424,0.347624,2021-12,NaN,Clifton HVAC,HVAC
611,101,911010570,0,12,2021-12-01,101238,FUSION HEATING AND COOLING LLC,PVC251,"3/4""X10' PVC SCH40 PIPE",EA,...,44625,18821.0,0.0,15.79500,9.34500,0.371718,2021-12,NaN,Clifton HVAC,HVAC
1000,101,911010708,0,1,2021-12-02,101233,G-W-HVAC,PVC251,"3/4""X10' PVC SCH40 PIPE",EA,...,44577,18821.0,0.0,26.32500,15.57500,0.371718,2021-12,NaN,Clifton HVAC,HVAC


In [12]:
sales_df.columns

Index(['whse', 'orderno', 'ordersuf', 'lineno', 'invoicedt', 'custno',
       'custname', 'item', 'itemdesc', 'unit', 'unitconv', 'units', 'unitcost',
       'replcost', 'unitprice', 'netamt', 'returnfl', 'transtype', 'prod_type',
       'prodcat', 'cat_descrip', 'rowpointer', 'priceorigcd', 'pdrecno',
       'vendno', 'arpvendno', 'xcost_adj', 'GP$', 'Margin', 'month',
       'Exclusion', 'whse_name', 'division'],
      dtype='object')

## Total sales by item and total customers

In [13]:
"""
group sales in two stages:
    first stage by 'item' and 'custno' with 'units' aggregated by sum
    second stage by 'item' aggregating 'custno' by count and 'units' by sum
the two stages of process let us count # of customers and get total sales of each item
"""
agg_sales_pre_df = sales_df.groupby(by=['item','itemdesc','custno'],as_index=False).agg({'units':'sum','netamt':'sum','xcost_adj':'sum'})

In [14]:
agg_sales_pre_df.to_clipboard(index=False)

In [15]:
agg_sales_pre_df

,item,itemdesc,custno,units,netamt,xcost_adj
0,CPF3000A,"1/2"" X 10' ""L"" COPPER",101013,24.0,619.68,475.36500
1,CPF3001,"3/4""X20' ""L"" COPPER PIPE",101013,11.0,1079.85,768.48918
2,CPF3001A,"3/4""X10' ""L"" COPPER PIPE",101013,115.0,5367.85,3905.64495
3,CPF3011,"3/4""X20' ""M"" COPPER PIPE",101013,8.0,501.04,311.77415
4,CPF3011,"3/4""X20' ""M"" COPPER PIPE",101035,2.0,128.66,60.89800
...,...,...,...,...,...,...
348,PVC50A,"2""X10' PVC SCH40 PIPE",110465,2.0,66.40,42.00000
349,PVC50A,"2""X10' PVC SCH40 PIPE",111261,2.0,62.12,42.65762
350,PVC50A,"2""X10' PVC SCH40 PIPE",111965,2.0,66.40,38.69048
351,PVC50A,"2""X10' PVC SCH40 PIPE",112441,3.0,91.80,57.20244


In [16]:
agg_sales_df = agg_sales_pre_df.groupby(by=['item','itemdesc'],as_index=False).agg({'custno':'count','units':'sum','netamt':'sum','xcost_adj':'sum'}).rename(columns={'custno':'total cust\'s','units':'total units'})

In [17]:
agg_sales_df['total margin'] = round( 100*(agg_sales_df['netamt'] - agg_sales_df['xcost_adj']) / agg_sales_df['netamt'], 2)

In [18]:
agg_sales_df = agg_sales_df.drop(columns=['netamt','xcost_adj'])

In [19]:
agg_sales_df.to_clipboard(index=False)

## Sales on overrides

In [20]:
overrides_df = sales_df.loc[sales_df['priceorigcd'] == 'O']

In [21]:
overrides_df.drop(columns=['ordersuf','lineno','unit','unitconv','arpvendno','month','Exclusion','cat_descrip','whse_name','division'])

,whse,orderno,invoicedt,custno,custname,item,itemdesc,units,unitcost,replcost,...,transtype,prod_type,prodcat,rowpointer,priceorigcd,pdrecno,vendno,xcost_adj,GP$,Margin
3757,101,911011538,2021-12-10,100088,A. CIAVAGLIA INC,PVC251,"3/4""X10' PVC SCH40 PIPE",40.0,5.26500,6.42,...,SO,stocked,7000,f3f70746-ba81-659b-8c14-43ae707054c5,O,0,18821.0,210.60000,107.40000,0.337736
16280,101,911015612,2022-02-01,101744,RB KRAFT LLC,PVC500,"3""X10' PVC SCH40 PIPE",1.0,20.71552,40.08,...,CS,stocked,7000,99af3962-7f75-29a0-8f14-6cf158c9ffd7,O,67052,18821.0,20.71552,36.54448,0.638220
19693,101,911016580,2022-02-11,101223,FERRETO HVAC,PVC50A,"2""X10' PVC SCH40 PIPE",1.0,21.32881,20.04,...,CS,stocked,7000,cda1f5ef-16b1-ed9a-9014-678e8838715c,O,9373,18821.0,21.32881,5.39119,0.201766
22202,101,911017337,2022-02-19,102774,ADVANCED MECHANICAL,PVC50A,"2""X10' PVC SCH40 PIPE",3.0,21.32881,20.04,...,CS,stocked,7000,9c7924c6-4f7a-cfab-9114-9f0bc86853a4,O,9373,18821.0,63.98643,11.16357,0.148550
31640,101,911020161,2022-03-21,105145,SANTIAGO'S HVAC SERVICES,PVC251,"3/4""X10' PVC SCH40 PIPE",4.0,6.33960,6.72,...,CS,stocked,7000,afba2a60-b0aa-6aa1-9214-86e3784a19e1,O,45489,18821.0,25.35840,11.20160,0.306389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260621,801,981010608,2022-10-03,101354,TOTAL COMFORT,CPF3110,"1/4"" X 50' REFRIG. COIL COPPER",1.0,40.06200,39.65,...,SO,stocked,6380,946f0448-d2f6-8d8a-9e14-7be8a00bb0f1,O,0,86125.0,40.06200,16.58800,0.292816
260980,801,981010706,2022-10-06,101013,BELL MECHANICAL LLC,CPF3001A,"3/4""X10' ""L"" COPPER PIPE",30.0,32.47000,30.49,...,SO,stocked,6380,93970650-8a09-47bd-9f14-aa0df06daa82,O,63025,86125.0,974.10000,477.90000,0.329132
261132,801,981010754,2022-10-07,101000,"A & G INSTALLATIONS, INC.",CPF3023,"3/4""ODX20' ""ACR"" COPPER PIPE",3.0,65.16406,65.15,...,SO,stocked,6380,e2cd53ed-fb64-049a-9f14-b72908705ab2,O,63023,86125.0,195.49218,77.53782,0.283990
262161,801,981011114,2022-10-20,101046,R.J.C. HEATING & COOLING,CPF3011A,"3/4""X10' ""M"" COPPER PIPE",2.0,24.91086,22.17,...,SO,stocked,6380,b08fc032-0573-0481-9f14-c8f5b8f74f11,O,63033,86125.0,49.82172,16.57828,0.249673


In [22]:
agg_overrides_df = overrides_df.groupby(by=['item'],as_index=False).agg({'orderno':'count','units':'sum'}).rename(columns={'orderno':'no. of override orders','units':'units overriden'})

In [23]:
agg_overrides_df

,item,no. of override orders,units overriden
0,CPF3000A,1,10.0
1,CPF3001,1,5.0
2,CPF3001A,5,95.0
3,CPF3011A,3,24.0
4,CPF3012A,1,2.0
5,CPF3013,1,1.0
6,CPF3023,13,50.0
7,CPF3110,3,3.0
8,CPF3112,11,80.0
9,CPF3114,1,2.0


## Total PT 7 sales (no overrides)

## <span style="color:red"> Change vendor name in last part of file path

In [24]:
loc = Path.cwd().parent / 'data' / 'merfish_PT_7_PRs.csv'
PT_7_PRs = pd.read_csv(loc)

In [25]:
PT_7_PRs = PT_7_PRs.drop_duplicates()

In [26]:
PT_7_sales_by_PR_df = sales_df.merge(PT_7_PRs[['prod','pdrecno']], how='inner', left_on=['item','pdrecno'], right_on=['prod','pdrecno'])

In [27]:
PT_7_sales_by_PR_df.head()

,whse,orderno,ordersuf,lineno,invoicedt,custno,custname,item,itemdesc,unit,...,vendno,arpvendno,xcost_adj,GP$,Margin,month,Exclusion,whse_name,division,prod
0,101,911007174,0,1,2022-03-04,100046,AIR PRO HEATING & COOLING,PVC500,"3""X10' PVC SCH40 PIPE",EA,...,18821.0,0.0,207.15430,299.94570,0.591492,2022-03,NaN,Clifton HVAC,HVAC,PVC500
1,101,911013280,0,6,2022-01-04,101164,CAP HVAC INC,PVC500,"3""X10' PVC SCH40 PIPE",EA,...,18821.0,0.0,103.57660,207.07340,0.666581,2022-01,NaN,Clifton HVAC,HVAC,PVC500
2,101,911014036,0,5,2022-01-13,102547,AIR ZONES,PVC500,"3""X10' PVC SCH40 PIPE",EA,...,18821.0,0.0,62.14656,124.24344,0.666578,2022-01,NaN,Clifton HVAC,HVAC,PVC500
3,101,911014994,0,2,2022-02-15,100278,ENVIRONMENTAL CLIMATE CONTRL,PVC500,"3""X10' PVC SCH40 PIPE",EA,...,18821.0,0.0,41.43104,82.82896,0.666578,2022-02,NaN,Clifton HVAC,HVAC,PVC500
4,101,911015612,0,1,2022-02-01,101744,RB KRAFT LLC,PVC500,"3""X10' PVC SCH40 PIPE",EA,...,18821.0,0.0,20.71552,36.54448,0.638220,2022-02,NaN,Clifton HVAC,HVAC,PVC500


In [28]:
PT_7_sales_by_PR_df.shape

(582, 34)

In [29]:
PT_7_sales_by_PR_df['margin by replcost'] = (PT_7_sales_by_PR_df['netamt'] - PT_7_sales_by_PR_df['replcost']*PT_7_sales_by_PR_df['units']) / PT_7_sales_by_PR_df['netamt']

In [30]:
# drop unnecessary columns
PT_7_sales_by_PR_df = PT_7_sales_by_PR_df.drop(columns=['orderno','ordersuf','lineno','unit','unitconv','returnfl','prod_type','prodcat','cat_descrip','rowpointer','arpvendno','Exclusion'])

In [31]:
# inspect output
PT_7_sales_by_PR_df.to_clipboard(index=False)

In [32]:
PT_7_no_overs_df = PT_7_sales_by_PR_df.loc[ PT_7_sales_by_PR_df['priceorigcd'] == '7']

In [33]:
# inspect output
PT_7_no_overs_df.to_clipboard(index=False)

In [34]:
"""
group sales in two stages:
    first stage by 'item' and 'custno' with 'units' aggregated by sum
    second stage by 'item' aggregating 'custno' by count and 'units' by sum
the two stages of process let us count # of customers and get total sales of each item
"""
PT_7_no_overs_agg_pre_df = PT_7_no_overs_df.groupby(by=['item','custno'],as_index=False).agg({'units':'sum'})

In [35]:
PT_7_no_overs_agg_pre_df.to_clipboard(index=False)

In [36]:
PT_7_no_overs_agg_pre_df.loc[PT_7_no_overs_agg_pre_df.duplicated(subset=['item','custno'])]

,item,custno,units


In [37]:
PT_7_no_overs_agg_df = PT_7_no_overs_agg_pre_df.groupby(by=['item'],as_index=False).agg({'custno':'count','units':'sum'}).rename(columns={'custno':'PT 7 cust\'s', 'units':'PT 7 units'})

In [38]:
PT_7_no_overs_agg_df

,item,PT 7 cust's,PT 7 units
0,CPF3011,2,9.0
1,CPF3020,1,1.0
2,CPF3023,1,1.0
3,CPF3024,1,1.0
4,CPF3110,1,1.0
5,CPF3114,1,1.0
6,CPF3116,1,10.0
7,PVC251,99,1171.0
8,PVC26,8,38.0
9,PVC401,1,29.0


In [39]:
# make cumulative percentage stats
PT_7_no_overs_quartiles_df = PT_7_no_overs_agg_pre_df[['item','units']].groupby('item').quantile(q=[.5,.75,1],interpolation='midpoint')

In [40]:
PT_7_no_overs_quartiles_df.head()

units
item               
CPF3011 0.50    4.5
        0.75    4.5
        1.00    8.0
CPF3020 0.50    1.0
        0.75    1.0

In [41]:
PT_7_no_overs_quartiles_df = PT_7_no_overs_quartiles_df.reset_index()

In [42]:
# data frame of quartiles of sales per customer
quartile_df = pd.DataFrame([ [prod,tuple(PT_7_no_overs_quartiles_df.loc[PT_7_no_overs_quartiles_df['item'] == prod]['units'].tolist())] for prod in PT_7_no_overs_quartiles_df['item'].unique()], columns=['item','PT 7 quartiles'])

In [43]:
quartile_df

,item,PT 7 quartiles
0,CPF3011,"(4.5, 4.5, 8.0)"
1,CPF3020,"(1.0, 1.0, 1.0)"
2,CPF3023,"(1.0, 1.0, 1.0)"
3,CPF3024,"(1.0, 1.0, 1.0)"
4,CPF3110,"(1.0, 1.0, 1.0)"
5,CPF3114,"(1.0, 1.0, 1.0)"
6,CPF3116,"(10.0, 10.0, 10.0)"
7,PVC251,"(4.0, 11.0, 238.0)"
8,PVC26,"(2.5, 4.0, 22.0)"
9,PVC401,"(29.0, 29.0, 29.0)"


# Standard cost, multiplier, and standard price

## <span style="color:red"> Change the vendor name at end of file path

In [44]:
loc = Path.cwd().parent / 'data' / 'merfish_product_info.csv'
cost_data_df = pd.read_csv(loc)

In [45]:
cost_data_df_subset = cost_data_df[['prod','stndcost','prcmult_1','stndprice']].drop_duplicates()

In [46]:
"""
make sure there are no mismatches in standard costs across warehouses
although warehouses are excluded in cost_data_df
there are multiplicities of entries corresponding to each warehouse
"""
cost_data_df_subset.loc[cost_data_df_subset.duplicated()]

,prod,stndcost,prcmult_1,stndprice


In [47]:
cost_data_df.columns

Index(['whse', 'vendno', 'name', 'prod', 'descrip_1', 'descrip_2', 'mfgprod',
       'vendprod', 'pricetype', 'prodcat', 'prodcat_descrip', 'unitstock',
       'unitsell', 'unitconv', 'listprice', 'avgcost', 'lastcost', 'replcost',
       'replcostdt', 'stndcost', 'stndcostdt', 'priceonty', 'prcmult_1',
       't12m_sales', 'qtyonhand', 'qtyonorder', 'stndprice'],
      dtype='object')

## <span style="color:red"> Change vendor number(s)

In [67]:
# pick out cost data for our vendor
specific_cost_data_df = cost_data_df.loc[ ((cost_data_df['vendno'] == 18821) | (cost_data_df['vendno'] == 86125)) ][['prod','descrip_1','descrip_2','stndcost','prcmult_1','stndprice']].drop_duplicates()

In [68]:
specific_cost_data_df['stndmargin'] = round((specific_cost_data_df['prcmult_1']*100 - 100) / specific_cost_data_df['prcmult_1'], 2)

In [69]:
specific_cost_data_df

,prod,descrip_1,descrip_2,stndcost,prcmult_1,stndprice,stndmargin
0,CPF3012,"1""X20' ""M"" COPPER PIPE",NaN,75.6800,2.310000,174.82,56.71
1,CPF3013A,"1-1/4""X10' ""M"" COPPER",PIPE,50.1800,1.000000,50.18,0.00
2,CPF3115,"3/4""X50' REFRIG. COIL",COPPER,126.6700,1.000000,126.67,0.00
3,CPF3001,"3/4""X20' ""L"" COPPER",PIPE,68.0900,1.000000,68.09,0.00
4,CPF3110,"1/4"" X 50' REFRIG. COIL",COPPER,39.6500,1.000000,39.65,0.00
5,CPF3022,"5/8""ODX20' ""ACR"" COPPER",PIPE,48.2900,1.000000,48.29,0.00
6,CPF3012A,"1""X10' ""M"" COPPER PIPE",NaN,37.8400,1.000000,37.84,0.00
7,CPF3021,"1/2""OD X 20' ""ACR",COPPER PIPE,37.6800,1.000000,37.68,0.00
8,CPF3112B,"3/8""X50' ORANGE COATED",NaN,60.5300,2.000000,121.06,50.00
9,BLK2101A,"3/4""X10' TC BLACK PIPE",NaN,18.0900,1.910000,34.55,47.64


## Put everything together

In [70]:
merge_1_df = agg_sales_df.merge(agg_overrides_df, how='left', on='item')

In [71]:
merge_2_df = merge_1_df.merge(PT_7_no_overs_agg_df, how='left', on='item')

In [72]:
merge_3_df = merge_2_df.merge(quartile_df, how='left', on='item')

In [73]:
merge_4_df = merge_3_df.merge(specific_cost_data_df, how='outer', left_on='item', right_on='prod')

In [85]:
final_df = merge_4_df.drop(columns='item')

In [86]:
final_df.columns

Index(['itemdesc', 'total cust's', 'total units', 'total margin',
       'no. of override orders', 'units overriden', 'PT 7 cust's',
       'PT 7 units', 'PT 7 quartiles', 'prod', 'descrip_1', 'descrip_2',
       'stndcost', 'prcmult_1', 'stndprice', 'stndmargin'],
      dtype='object')

In [87]:
final_df.head()

,itemdesc,total cust's,total units,total margin,no. of override orders,units overriden,PT 7 cust's,PT 7 units,PT 7 quartiles,prod,descrip_1,descrip_2,stndcost,prcmult_1,stndprice,stndmargin
0,"1/2"" X 10' ""L"" COPPER",1.0,24.0,23.29,1.0,10.0,NaN,NaN,NaN,CPF3000A,"1/2"" X 10' ""L"" COPPER",NaN,19.88,1.0,19.88,0.0
1,"3/4""X20' ""L"" COPPER PIPE",1.0,11.0,28.83,1.0,5.0,NaN,NaN,NaN,CPF3001,"3/4""X20' ""L"" COPPER",PIPE,68.09,1.0,68.09,0.0
2,"3/4""X10' ""L"" COPPER PIPE",1.0,115.0,27.24,5.0,95.0,NaN,NaN,NaN,CPF3001A,"3/4""X10' ""L"" COPPER",PIPE,30.49,1.0,30.49,0.0
3,"3/4""X20' ""M"" COPPER PIPE",3.0,11.0,41.77,NaN,NaN,2.0,9.0,"(4.5, 4.5, 8.0)",CPF3011,"3/4""X20' ""M"" COPPER",PIPE,49.52,1.0,49.52,0.0
4,"3/4""X10' ""M"" COPPER PIPE",2.0,41.0,19.66,3.0,24.0,NaN,NaN,NaN,CPF3011A,"3/4""X10' ""M"" COPPER",PIPE,22.17,1.0,22.17,0.0


In [88]:
final_df = final_df[['prod','descrip_1','descrip_2','total cust\'s', 'total units', 'total margin', 'no. of override orders', 'units overriden', 'PT 7 cust\'s', 'PT 7 units', 'PT 7 quartiles', 'stndcost', 'prcmult_1', 'stndprice', 'stndmargin']]

## Save records of sales

## <span style="color:red"> Change vendor name at end of file path

In [89]:
loc = Path.cwd().parent / 'data' / 'merfish_sales_record.csv'
final_df.to_csv(loc, index=False)